In [1]:
import os
import pickle
import pandas as pd

In [2]:
csv_file_path = '/home/nogaschw/Codeworkout/cleaned_code.csv'
cwd = os.getcwd()

df = pd.read_csv(csv_file_path, sep=',')
df = df[df.columns[1:]]


In [3]:
cleaned_file_path = '/home/nogaschw/Codeworkout/cleaned_questions.csv'

questions = pd.read_csv(cleaned_file_path, sep=',')
questions = questions[questions.columns[1:]]
questions = questions[questions['max_score'] != 0]

In [4]:
df['problem_id'] = df['problem_id'].apply(lambda x: x.lower())
questions['id'] = questions['id'].apply(lambda x: x.lower())

In [5]:
df = df.sort_values('timestamp', ascending=True)
df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({
    'ServerTimestamp': x['timestamp'].tolist(),
    'score': x['score'].tolist(),
    'source_code': x['clean_code'].tolist()
})).reset_index()

/tmp/ipykernel_895104/2430072954.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({


In [6]:
def trim_after_max(score, snapshots, ServerTimestamp):
    max_index = len(score) - 1 - score[::-1].index(max(score))
    return score[:max_index+1], snapshots[:max_index+1], ServerTimestamp[:max_index+1]

# Apply the function to each row
df['score'], df['source_code'], df['ServerTimestamp'] = zip(*df.apply(lambda row: trim_after_max(row['score'], row['source_code'], row['ServerTimestamp']), axis=1))
df['struggling'] = df['score'].apply(lambda x: 0 if x[-1] == 100 else 1)
df['start_time'] = [i[0] for i in df['ServerTimestamp']]
df['max_score'] = [max(i) for i in df['score']]

In [7]:
len(df[df['max_score'] == -1])
df = df[df['max_score'] != -1] # remove all the rows contain only locall code (not really the grade)

In [8]:
df['struggling'].value_counts()

struggling
0    115052
1     27659
Name: count, dtype: int64

In [9]:
27659 / 115052

0.2404043389076244

In [10]:
df_basic = df.merge(questions, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [11]:
df_basic = df_basic[['student_id', 'problem_id', 'course_id', 'start_time', 'score', 'source_code', 'prompt', 'struggling']]

In [12]:
df_sorted = df_basic.sort_values(by=['student_id', 'start_time'])
df_sorted['row_number'] = df_sorted.groupby('student_id').cumcount() + 1

In [13]:
prev_code = df_sorted['source_code'].tolist()
prev_code.insert(0, [])
prev_q = df_sorted['prompt'].tolist()
prev_q.insert(0, [])
prev_num_q = df_sorted['problem_id'].tolist()
prev_num_q.insert(0, [])
prev_score = df_sorted['score'].tolist()
prev_score.insert(0, [])

In [14]:
df_sorted['prev_code'] = prev_code[:-1] # last question of the last student
df_sorted['prev_question'] = prev_q[:-1]
df_sorted['prev_question_num'] = prev_num_q[:-1]
df_sorted['score'] = prev_score[:-1]

df_sorted = df_sorted[df_sorted['row_number'] != 1]

In [15]:
prev_code = []
prev_question = []
prev_q_num = []
prev_score = []
for i, row in df_sorted.iterrows():
    if row['row_number'] == 2:
        prev_code.append([row['prev_code']])
        prev_question.append([row['prev_question']])
        prev_q_num.append([row['prev_question_num']])
        prev_score.append([row['score']])
    else:
        prev_code.append(prev_code[-1] + [row['prev_code']])
        prev_question.append(prev_question[-1] + [row['prev_question']])
        prev_q_num.append(prev_q_num[-1] + [row['prev_question_num']])
        prev_score.append(prev_score[-1] + [row['score']])

In [16]:
df_sorted['prev_code'] = prev_code
df_sorted['prev_question'] = prev_question
df_sorted['prev_question_num'] = prev_q_num
df_sorted['score'] = prev_score

# Taxonomy

In [17]:
q_with_compu = questions.copy()
q_with_compu['IfElse'] = questions['conditional']
q_with_compu['Loops'] = questions[["loop_counting", "loop_until", "loop_elements", "loop_nested"]].max(axis=1)
q_with_compu['MathOperations'] = questions[["stat_calculate", "assignment"]].max(axis=1)
q_with_compu['LogicOperators'] = 0
q_with_compu['StringOperations'] = questions[["input_str", "input_cast", "output"]].max(axis=1)
q_with_compu["List"]= questions[["list", "list_2d"]].max(axis=1)
q_with_compu['FileOperations'] = questions[["file_read", "file_write"]].max(axis=1)
q_with_compu["Functions"] = questions[["function_call", "function_def", "function_return"]].max(axis=1)
q_with_compu['Dictionary'] = questions["dictionary"]
q_with_compu['Tuple'] = questions[["tuple", "item_set"]].max(axis=1)

q_with_compu = q_with_compu.drop(columns=['input_str', 'input_cast', 'output', 'assignment', 'conditional',
       'function_call', 'function_def', 'function_return', 'loop_counting',
       'loop_until', 'loop_elements', 'loop_nested', 'stat_calculate',
       'file_read', 'file_write', 'list', 'list_2d', 'dictionary', 'item_set',
       'tuple'])


q_with_compu.fillna(0, inplace=True)
questions.fillna(0, inplace=True)

In [18]:
q_with_compu = q_with_compu.rename(columns={'prompt': 'question'})

In [19]:
df_taxonomy = df_sorted.merge(q_with_compu, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [20]:
df_taxonomy = df_taxonomy[['student_id', 'course_id', 'prev_code', 'prev_question', 'question', 'struggling', 'type', 'IfElse', 'Loops',
       'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple']]

In [21]:
with open(f'/home/nogaschw/Codeworkout/Thesis/Data/falcon/split_ids_valid_ids.pkl', 'rb') as f:
        valid_ids = pickle.load(f)
with open(f'/home/nogaschw/Codeworkout/Thesis/Data/falcon/split_ids_test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

In [22]:
df_taxonomy = df_taxonomy[df_taxonomy['type'] != 'project']

In [23]:
valid_df = df_taxonomy[df_taxonomy['student_id'].isin(valid_ids)]
test_df = df_taxonomy[df_taxonomy['student_id'].isin(test_ids)]

In [31]:
pd.concat([valid_df, test_df], ignore_index=True).to_pickle(os.path.join(cwd, '../df_taxonomy_falcon_test_valid.pkl'))

In [23]:
max_snapshots = 0
max_total = 0
max_row_snapshots = None
max_row_total = None
for j, row in df_taxonomy.iterrows():
    r = row['prev_code']
    sum_snapshots = 0
    for i in r:
        sum_snapshots += len(i)
        if len(i) > max_snapshots:
            max_snapshots = len(i)
            max_row_snapshots = row
    if sum_snapshots > max_total:
        max_total = sum_snapshots
        max_row_total = row


In [24]:
max_total

2109

In [25]:
max_snapshots

290

In [26]:
from Data import *
from transformers import AutoTokenizer

/home/nogaschw/.conda/envs/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
hidden_size = 128
num_layers = 2
lr = 0.00001
all_questions = 1
taxonomy = 0
balance_def = 1
mean = 1

balance_def = [oversampling_boosting, oversampling_df, undersampling_df, oversampling_augmentation][balance_def]

name = f'{["", "Mean_"][mean]}{"All_History_with"}_{["Current_Question", "All_Question", "FullyConnected"][all_questions]}_{hidden_size}_{num_layers}_{lr}_{balance_def.__name__}'

dataset = [DatasetCodeQuestion, DatasetCodeQPrevQ, DatasetLimitMean][all_questions]

feature = None

text_model_name = 'google-bert/bert-base-uncased'
code_model_name = 'microsoft/codebert-base'

text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
code_tokenizer = AutoTokenizer.from_pretrained(code_model_name)
if text_tokenizer.pad_token is None:
    text_tokenizer.pad_token = text_tokenizer.eos_token
if code_tokenizer.pad_token is None:
    code_tokenizer.pad_token = code_tokenizer.eos_token

In [42]:
train_dataloader, valid_dataloader, test_dataloader = create_data_loader(df_taxonomy, DatasetCodeQuestion, text_tokenizer, code_tokenizer, batch_size=32, balanced_def=balance_def, feature=feature, ids_filepath_prefix="/home/nogaschw/Codeworkout/Thesis/Data/falcon/split_ids")

Load exist spliting


In [43]:
# Check data leak
print(set(train_dataloader.dataset.df['student_id']).intersection(set(test_dataloader.dataset.df['student_id'])))
print(set(valid_dataloader.dataset.df['student_id']).intersection(set(test_dataloader.dataset.df['student_id'])))
print(set(valid_dataloader.dataset.df['student_id']).intersection(set(train_dataloader.dataset.df['student_id'])))

set()
set()
set()
